In [ ]:
import igl
import numpy as np
from scipy.io import savemat
import os
from PyRMT import RMTMesh
import sys


sys.path.append('../')
from visual_utils import *
from icp_utils import *

#from evaluation import *

def visu(vertices):
    
    "The function normalizes the values over the vertices"
    
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap



In [ ]:
from pyFM.mesh import TriMesh
target_re = TriMesh(r'D:\Make_Galileo_great_again\data\giorgio_\MNI_remesh.off')
target_hr = TriMesh(r'D:\Make_Galileo_great_again\data\giorgio_\MNI.off', area_normalize=True, center= True)


In [ ]:
plot_mesh(target_hr)

In [ ]:
target_re.vertlist.shape

In [ ]:
SamplesDir = r"D:\Make_Galileo_great_again\data\giorgio_"

N_meshes = 0
m_list=[]
m_rem_list=[]
bm_list=[]

for i in range(N_meshes):
    v, f = igl.read_triangle_mesh(os.path.join(SamplesDir, fr"0000{i:02d}_tumoredbrain\0000{i:02d}_tumoredbrain.off"))
    v = np.asfortranarray(v)
    f = np.asfortranarray(f)

    m = RMTMesh(v, f)
    m.make_manifold()
    m_list.append(m)

    m_rem = m.remesh(6000)
    m_rem.clean_up()
    m_rem_list.append(m_rem)

    bm = m_rem.baryc_map(v)
    bm_list.append(bm)
    
    igl.write_triangle_mesh(os.path.join(SamplesDir, fr"00000{i:02d}_tumoredbrain.off"), m_rem.vertices, m_rem.triangles)

v, f = igl.read_triangle_mesh(r"D:\Make_Galileo_great_again\data\giorgio_\MNI.off")
v = np.asfortranarray(v)
f = np.asfortranarray(f)

m = RMTMesh(v, f)
m.make_manifold()
m_rem = m.remesh(6000)
m_rem.clean_up()
bm = m_rem.baryc_map(v)
igl.write_triangle_mesh(os.path.join(SamplesDir, "MNI_remesh.off"), m_rem.vertices, m_rem.triangles)
savemat(os.path.join(SamplesDir, "MNI_remesh.mat"), {"U" : bm})

In [ ]:
from pyFM.mesh import TriMesh

m0= TriMesh(m_list[0].vertices, m_list[0].triangles)
m0_rem=TriMesh(m_rem_list[0].vertices, m_rem_list[0].triangles)
m1 = TriMesh(m_list[1].vertices, m_list[1].triangles)
m1_rem = TriMesh(m_rem_list[1].vertices, m_rem_list[1].triangles)
m2 = TriMesh(m_list[2].vertices, m_list[2].triangles)
m2_rem = TriMesh(m_rem_list[2].vertices, m_rem_list[2].triangles)

In [ ]:
p2p_KNN = knn_query_normals(m0.vertlist, m0_rem.vertlist,
                                            m0.vertex_normals, m0_rem.vertex_normals,
                                            k_base=20, n_jobs=20, verbose=False)
#p2p_KNN = knn_query(meshlist[0].vertlist, meshlist[1].vertlist)

In [ ]:
plot_mesh(m0, color=np.mean(m0.vertices,axis=1))
plot_mesh(m0_rem, color=np.mean(m0.vertices,axis=1)[p2p_KNN])

KNN

In [ ]:
p2p_KNN = knn_query_normals(m0.vertlist, m0_rem.vertlist,
                                            m0.vertex_normals, m0_rem.vertex_normals,
                                            k_base=20, n_jobs=20, verbose=False)
#p2p_KNN = knn_query(meshlist[0].vertlist, meshlist[1].vertlist)

In [ ]:

p2p_KNN = knn_query(m0_rem.vertlist, m1_rem.vertlist)

In [ ]:
plot_mesh(m0_rem, color=np.mean(m0_rem.vertices,axis=1))
plot_mesh(m1_rem, color=np.mean(m0_rem.vertices,axis=1)[p2p_KNN])

In [ ]:
#p2p_KNN = knn_query_normals(m0_rem.vertlist, m2_rem.vertlist,
#                                            m0_rem.vertex_normals, m2_rem.vertex_normals,
#                                            k_base=20, n_jobs=20, verbose=False)
p2p_KNN = knn_query(m0_rem.vertlist, m2_rem.vertlist)

In [ ]:
plot_mesh(m0_rem, color=np.mean(m0_rem.vertices,axis=1))
plot_mesh(m2_rem, color=np.mean(m0_rem.vertices,axis=1)[p2p_KNN])

ZOOMOUT

In [ ]:
import pyFM.spectral as spectral
from pyFM.FMN import FMN
import pyFM

In [ ]:
from pyFM.refine.zoomout import mesh_zoomout_refine_p2p

Calculate the Laplacian basis

In [ ]:
m0_rem.process(k=150,intrinsic=True)
m2_rem.process(k=150,intrinsic=True)

Apply ZoomOut

In [ ]:
# apply zoomout starting from the Knn initialization
FM_12_zo, p2p_ZO = mesh_zoomout_refine_p2p(p2p_21=p2p_KNN, mesh1=m0_rem, mesh2=m2_rem, k_init=20, nit=26, step=5, return_p2p=True, n_jobs=10, verbose=True)

plot_mesh(m0_rem,color=np.mean(m0_rem.vertices,axis=1))
plot_mesh(m2_rem,color=np.mean(m0_rem.vertices,axis=1)[p2p_ZO])

Import original meshes

In [ ]:
#1 min e 11 sec.
import trimesh
#Number of meshes to import:
N_meshes= 3
meshlist=[]
for i in range(N_meshes):
    meshlist.append(TriMesh(fr'D:\Make_Galileo_great_again\data\giorgio_\0000{i:02d}_tumoredbrain\0000{i:02d}_tumoredbrain.off')) 
    mesh= trimesh.Trimesh(meshlist[i].vertices, meshlist[i].faces)
    components = mesh.split(only_watertight=False)
    largest_component = max(components, key=lambda comp: comp.volume) 
    print(f"Number of vertices before LC for mesh{i:02d}:", meshlist[i].vertices.shape)
    #meshlist[i]= TriMesh(largest_component.vertices, largest_component.faces).process(k=200, intrinsic=True)
    #print(f"Number of vertices after LC for mesh{i:02d}:", meshlist[i].vertices.shape)


Correspondence with upscaled FMap + Transferring function


In [ ]:
p2p_RM = spectral.FM_to_p2p(FM_12_zo, bm_list[0] @ m0_rem.eigenvectors, bm_list[2] @ m2_rem.eigenvectors, n_jobs=10)

Plot

In [ ]:
plot_mesh(meshlist[0],color=np.mean(meshlist[0].vertices,axis=1))
plot_mesh(meshlist[2],color=np.mean(meshlist[0].vertices,axis=1)[p2p_RM])

## Rematching with FunctionalMaps

### ICP

In [ ]:
subsample_list = np.zeros((3, 10), dtype=int)
for i in tqdm(range(3)):
    subsample_list[i] = meshlist[i].extract_fps(10, geodesic=False, verbose=False)
fps1 = subsample_list[0]
fps2 = subsample_list[2]

In [ ]:
# Get initial correspondences with KNN
p2p_21_init_sub = knn_query_normals(meshlist[0].vertlist[fps1], meshlist[2].vertlist[fps2],
                                            meshlist[0].vertex_normals[fps1], meshlist[2].vertex_normals[fps2],
                                            k_base=5, n_jobs=20, verbose=False)
# ICP Align the shape
_, R, t = icp_align(meshlist[2].vertlist[fps2], meshlist[0].vertlist[fps1],
                            p2p_12=p2p_21_init_sub,
                            return_params=True, n_jobs=20, epsilon=1e-4, verbose=False)

meshlist[2].rotate(R)
meshlist[2].translate(t)


### Rematching

In [ ]:
rem_list_tri=[m0_rem, m1_rem, m2_rem]

SRC=0
TRG=2


In [ ]:
subsample_list = np.zeros((3, 10), dtype=int)
for i in tqdm(range(3)):
    subsample_list[i] = rem_list_tri[i].extract_fps(10, geodesic=False, verbose=False)

KNN

In [ ]:
fps2 = knn_query(rem_list_tri[TRG].vertlist, rem_list_tri[SRC].vertlist[subsample_list[SRC]])

landmarks=np.array([subsample_list[SRC],fps2]).T

### Functional Maps

In [ ]:
from pyFM.functional import FunctionalMapping

process_params = {
        'n_ev': (20,20),  # Number of eigenvalues on source and Target
        'landmarks': landmarks,
        'subsample_step': 1,  # In order not to use too many descriptors
        'n_descr': 40, #number of descriptors
        'descr_type': 'HKS',  # WKS or HKS
    }
model = FunctionalMapping(rem_list_tri[SRC],rem_list_tri[TRG])
model.preprocess(**process_params,verbose=False)

model.fit(w_descr= 1e-1, w_lap= 1e-3, w_dcomm= 1,w_orient= 0, verbose=False)
fmap12=model.FM       #C{XY}, or A_{YX}^T

#p2p=model.get_p2p(fmap12, model.mesh1.eigenvectors[:,:k],model.mesh2.eigenvectors[:,:k],adj, bijective)
p2p=model.get_p2p()


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(fmap12)

In [ ]:
#cmap1 = np.mean(visu(rem_list_tri[SRC].vertlist),axis=1)
#cmap2_wks = cmap1[p2p]

plot_mesh(rem_list_tri[SRC],color=np.mean(rem_list_tri[SRC].vertices,axis=1))
plot_mesh(rem_list_tri[TRG],color=np.mean(rem_list_tri[SRC].vertices,axis=1)[p2p])

In [ ]:
#zoomout
from pyFM.refine.zoomout import mesh_zoomout_refine_p2p
FM_12_hks_zo, p2p_21_hks_zo = mesh_zoomout_refine_p2p(p2p_21=p2p, mesh1=rem_list_tri[SRC], mesh2=rem_list_tri[TRG], k_init=20, nit=16, step=5, return_p2p=True, n_jobs=10, verbose=True)

#cmap1 = np.mean(visu(rem_list_tri[SRC].vertlist),axis=1)
#cmap2_wks = cmap1[p2p_21_wks_zo]


plot_mesh(rem_list_tri[SRC],color=np.mean(rem_list_tri[SRC].vertices,axis=1))
plot_mesh(rem_list_tri[TRG],color=np.mean(rem_list_tri[SRC].vertices,axis=1)[p2p_21_hks_zo])

In [ ]:
plt.imshow(FM_12_hks_zo)

In [ ]:
p2p_ZO_RM = spectral.FM_to_p2p(FM_12_hks_zo, bm_list[SRC] @ rem_list_tri[SRC].eigenvectors, bm_list[TRG] @ rem_list_tri[TRG].eigenvectors, n_jobs=10)

In [ ]:
plot_mesh(meshlist[SRC],color=np.mean(meshlist[SRC].vertices,axis=1))
plot_mesh(meshlist[TRG],color=np.mean(meshlist[SRC].vertices,axis=1)[p2p_ZO_RM])

### CRANIOFACIAL

In [ ]:
import sys

sys.path.append(r'D:\Make_Galileo_great_again')
from utils.Morpho import *

sys.path.append('./pyFM/')

import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
import itertools
import copy

import networkx as nx

import pyFM.spectral as spectral
from pyFM.mesh import TriMesh
from pyFM.FMN import FMN

from tqdm.auto import tqdm
import seaborn as sns

SamplesDir = r"D:\Make_Galileo_great_again\data\giorgio_"
maps_dict={}
N_meshes=51

m_list= []
m_rem_list= []
bm_list= []

#meshes in Trimesh
meshlist= []
rem_list_tri = []

#Dictionary where Fun Maps are stored.
maps_dict = {}

for i in range(N_meshes):
    
    PATH = os.path.join(SamplesDir, "remeshed", fr"0000{i:02d}_tumoredbrain.off")
    print(fr"0000{i:02d}_tumoredbrain\0000{i:02d}_tumoredbrain.off")
    PATH_hr = os.path.join(SamplesDir, fr"0000{i:02d}_tumoredbrain\0000{i:02d}_tumoredbrain.off")
    PATH_bm = os.path.join(SamplesDir, "bm", fr"bm{i:02d}.mat")

    if os.path.exists(PATH):

        rem_list_tri.append(TriMesh(PATH, area_normalize=True, center= True).process(k=110, intrinsic=True))
        meshlist.append(TriMesh(PATH_hr, area_normalize=True, center= True))
    #print("Mesh" , i, "loaded")

target_hr = TriMesh(r'D:\Make_Galileo_great_again\data\giorgio_\MNI.off', area_normalize=True, center= True)
meshlist.append(target_hr)
target_re = TriMesh(r'D:\Make_Galileo_great_again\data\giorgio_\MNI_remesh.off', area_normalize=True, center= True).process(k=110, intrinsic=True)
rem_list_tri.append(target_re)

#for i in range (N_meshes):
#    PATH_SAVE= os.path.join(SamplesDir, fr"FM\fmap12{i:02d}.npy")
#    FM_12_hks_zo=np.load(PATH_SAVE)
#    maps_dict[3,i]= FM_12_hks_zo.copy()
#    maps_dict[i,3]= FM_12_hks_zo.T



Test


In [ ]:

n_chosen_pairs = 2500  # Number of pairs to select in the network
k_init = 20  # Initial size of functional maps. We recommand around 20 but because of the double surface we need 50
n_subsample = 3000 # Number of samples to use for initial maps

print(f'{int(scipy.special.binom(N_meshes, 2)):d} possible pairs')

In [ ]:
rng = np.random.default_rng()

all_pairs = list(itertools.combinations(np.arange(N_meshes), 2))
rng.shuffle(all_pairs)

chosen_pairs = all_pairs[:n_chosen_pairs]

G = nx.Graph()
G.add_nodes_from(np.arange(N_meshes))
G.add_edges_from(chosen_pairs)


print(f'Is G connected ? {nx.is_connected(G)}')
all_cliques= nx.enumerate_all_cliques(G)
triad_cliques=[x for x in all_cliques if len(x)==3 ]
print(f'G has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges')
print(f'G has {len(triad_cliques)} 3-cycles')

In [ ]:
subsample_list = np.zeros((N_meshes, n_subsample), dtype=int)
for i in tqdm(range(len(rem_list_tri)-1)):
    print(i)
    subsample_list[i] = rem_list_tri[i].extract_fps(n_subsample, geodesic=False, verbose=False)
    

In [ ]:
maps_dict = {}

for pair_ind, (i, j) in enumerate(tqdm(chosen_pairs)):
    
    
    fps1 = subsample_list[i]
    fps2 = subsample_list[j]
    
       

    mesh1 = copy.deepcopy(rem_list_tri[i])
    mesh2 = copy.deepcopy(rem_list_tri[j])

    # Get initial correspondences
    p2p_21_init_sub = knn_query_normals(mesh1.vertlist[fps1], mesh2.vertlist[fps2],
                                                mesh1.vertex_normals[fps1], mesh2.vertex_normals[fps2],
                                                k_base=20, n_jobs=10, verbose=False)
    
    # ICP Align the shape
    _, R, t = icp_align(mesh2.vertlist[fps2], mesh1.vertlist[fps1],
                                p2p_12=p2p_21_init_sub,
                                return_params=True, n_jobs=10, epsilon=1e-4, verbose=False)

    mesh2.rotate(R);
    mesh2.translate(t);
    
    # Get final correspondences
    p2p_21 = knn_query_normals(mesh1.vertlist[fps1], mesh2.vertlist[fps2],
                                        mesh1.vertex_normals[fps1], mesh2.vertex_normals[fps2],
                                        k_base=20, n_jobs=10)
    
    p2p_12 = knn_query_normals(mesh2.vertlist[fps2], mesh1.vertlist[fps1],
                                        mesh2.vertex_normals[fps2], mesh1.vertex_normals[fps1],
                                        k_base=20, n_jobs=10)
        
    # Compute initial functional maps
    FM_12 = spectral.mesh_p2p_to_FM(p2p_21, rem_list_tri[i], rem_list_tri[j], dims=k_init, subsample=(fps1, fps2))
    FM_21 = spectral.mesh_p2p_to_FM(p2p_12, rem_list_tri[j], rem_list_tri[i], dims=k_init, subsample=(fps2, fps1))
    maps_dict[(i,j)] = FM_12.copy();
    maps_dict[(j,i)] = FM_21.copy();
    
print(f'{len(maps_dict.keys())} maps computed');


In [ ]:
network = FMN(rem_list_tri[:11], maps_dict=maps_dict)

In [ ]:
czo_parameters = {
    'nit': (110-k_init)//5,
    'step': 5,
    'cclb_ratio': .8,
    'subsample': subsample_list,
    'isometric': False,
    'verbose': True,
    'weight_type': 'iscm',
    'n_jobs': 15,
}


network.zoomout_refine(**czo_parameters)

network.compute_W(M=110)
network.compute_CCLB(int(.8*network.M), verbose=True)

In [ ]:
network.compute_W(M=105)
network.compute_CCLB(int(.8*network.M), verbose=True)

In [ ]:
network.cclb_eigenvalues[:,None][0]=0

In [ ]:
deviation_from_id_a = np.zeros(network.n_meshes)
deviation_from_id_cr = np.zeros(network.n_meshes)
for i in range(network.n_meshes):
    CSD_a, CSD_c = network.get_CSD(i)
    deviation_from_id_a[i] = np.linalg.norm(CSD_a - np.eye(CSD_a.shape[0]))
    deviation_from_id_cr[i] = np.linalg.norm(np.sqrt(network.cclb_eigenvalues[:,None]) * (CSD_c - np.eye(CSD_c.shape[0]))) / np.sqrt(network.cclb_eigenvalues.sum())

    
deviation_from_id_cr.argmin(), deviation_from_id_cr.argmin()


In [ ]:
base_meshind = deviation_from_id_cr.argmin()

In [ ]:
k_displacement = 100
backend = 'gpu' # OR "cpuù
from pyFM.FMN import FMN

In [ ]:
LB_2.shape

In [ ]:
import pyFM
mesh1 = copy.deepcopy(network.meshlist[base_meshind]) # TriMesh(network.meshlist[base_meshind].path, area_normalize=True).process(k=k_displacement,verbose=True)
LB_1 = network.get_LB(base_meshind, complete=True)  # (n_1',m)

displacements = np.zeros((network.n_meshes, 3*mesh1.n_vertices))
displacements_red = np.zeros((network.n_meshes, 3*k_displacement))

for meshind2 in tqdm(range(network.n_meshes)):
    if meshind2 == base_meshind:
        continue
    
    mesh2 = copy.deepcopy(network.meshlist[meshind2])
    LB_2 = network.get_LB(meshind2, complete=True)  # (n_2',m)
    
    #p2p_czo_12 = knn_query(torch.from_numpy(LB_2.astype(np.float32)).cuda(), torch.from_numpy(LB_1.astype(np.float32)).cuda(), backend=backend).cpu().numpy()

    p2p_czo_12 = knn_query(LB_2.astype(np.float32), LB_1.astype(np.float32))

    R, t = rigid_alignment(mesh1.vertlist, mesh2.vertlist, p2p_12=p2p_czo_12,
                                 return_params=True, return_deformed=False, weights=mesh1.vertex_areas)
    
    
    mesh2 = TriMesh(mesh2.vertlist, mesh2.facelist)
    mesh2.translate(-t)
    mesh2.rotate(np.linalg.inv(R))
    
    tau_czo = mesh1.project(mesh2.vertlist[p2p_czo_12] - mesh1.vertlist, k=k_displacement)
    
    displacements[meshind2] = mesh1.decode(tau_czo).flatten()
    displacements_red[meshind2] = tau_czo.flatten()
print('');

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA

In [ ]:
avg_disp = np.mean(displacements_red,axis=0)
pca_d = PCA(n_components=5)
emb_d_red = pca_d.fit_transform(displacements_red - avg_disp[None,:])

plt.figure(dpi=150)
plt.title('Explained variance ratio')
plt.xlabel('Number of components')
plt.ylabel('Explained variance (%)')
plt.plot(np.arange(1+pca_d.n_components) , 100*np.cumsum(np.concatenate([[0],pca_d.explained_variance_ratio_])), marker=".");

In [ ]:
labels = [1,2] # list of labels for each skulls
labels = np.asarray(labels)

In [ ]:

reglin1 = LogisticRegression(penalty='none', fit_intercept=True, max_iter=1000)

In [ ]:
reglin1.fit()